In [1]:
import pandas as pd
import os
import requests
import json
from dotenv import load_dotenv


pd.set_option('display.max_columns', None)

Variables List + Header API

In [2]:
load_dotenv()
RIOT_API_KEY = os.getenv("RIOT_API_KEY")
my_game_name = os.getenv("my_game_name")
my_tag_line = os.getenv("my_tag_line")

In [4]:
headers = {
    "X-Riot-Token" : RIOT_API_KEY
}

# Creating a quick data frame just for the purpose of testing the functions in the blocks below

In [4]:
api_url = "https://na1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5"

resp = requests.get(api_url, headers=headers)
response = resp.json()
entries = response.get('entries', [])
entries = entries[:4]
test_df = pd.DataFrame([{"summonerId" : entry["summonerId"]} for entry in entries])
test_df.to_csv("test.csv", index = False)

# Function to read csv above and populate it (and the dataframe) with puuids in a new column

In [5]:
def get_puuid(csv_only_summ_ids):
    # Assigning csv file here as a placeholder, can just include it in the argument when using this fucntion
    df_only_summ_ids = pd.read_csv(csv_only_summ_ids, index_col=False)
    # Loop to iterate through each row
    for index, row in df_only_summ_ids.iterrows():
            summoner_id = row["summonerId"]
            # Variable to insert summ_id in API url that fetches account data
            api_url = f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}"
            try:
                resp = requests.get(api_url, headers=headers)  #Insert API link here
                resp.raise_for_status()
                summoner_info = resp.json()
                # Create a column in the data frame called puuids and populate it with the puuid for each summoner id
                df_only_summ_ids.at[index, "puuid"] = summoner_info.get("puuid", "Not Found") # Store puuid in df, output Not Found if KeyError
            # Alternative way to handle request exceptions that handles all request Errors instead of just error 400
            except requests.RequestException as e:
                print(f"Error fetching puuid for {summoner_id}: {e}")
    # Update csv file with the new column
    df_only_summ_ids.to_csv(csv_only_summ_ids, index = False)
# Calling function to add new info to existing csv instead of creating new one (using test csv from above in this case)
get_puuid("test.csv")

# Function to get match ids using puuids from Dataframe

In [11]:
def get_match_ids(csv_summids_and_puuids, current_patch):
    # Read csv file with summoner ids and puuids 
    df_summoners = pd.read_csv(csv_summids_and_puuids)
    # Create a new DataFrame to store match data
    df_all_matches = pd.DataFrame(columns = ["summonerId", "puuid", "matchId", "matchData"])
    # Variable for number of matches, will be defaulted to max (100) in actual code, small number used for testing
    number_of_matches = 5
    # Iterate through each row in df_summoners
    for _, row in df_summoners.iterrows():
        summoner_id = row["summonerId"]
        puuid = row["puuid"]
        # Fetch match ids for each puuid
        match_history_api_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count={number_of_matches}"

        try:
            resp = requests.get(match_history_api_url, headers=headers)
            resp.raise_for_status()
            match_ids = resp.json()
            # Iterate through each match id and fetch match data
            for match_id in match_ids:
                match_data_api_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}"
                response = requests.get(match_data_api_url, headers=headers)
                match_data = response.json()
                # Variable to check for patch/version
                game_patch = (match_data["info"]["gameVersion"].split("."))[0] + "." + (match_data["info"]["gameVersion"].split("."))[1]
                if game_patch == current_patch:
                    # Store match data in JSON file along with summoner id and puuid
                    df_mini = pd.DataFrame({
                        "summonerId" : [summoner_id],
                        "puuid" : [puuid],
                        "matchId" : [match_id],
                        "matchData": [json.dumps(match_data)]
                        })
                    # Append to main DataFrame
                    df_all_matches = pd.concat([df_all_matches, df_mini], ignore_index = True)
                else:
                    continue
        # Alternative way to handle request exceptions that handles all request Errors instead of just error 400
        except requests.RequestException as e:
            print(f"Error fetching matches for {puuid}: {e}")
    df_all_matches = df_all_matches.drop_duplicates("matchId")
    # Save the expanded DataFrame
    df_all_matches.to_csv("all_match_data_test.csv", index = False)
    print(df_all_matches.nunique())
    return df_all_matches

In [12]:
# Running get_match function, argument 1 = csv file, argument 2 = current patch
get_match_ids("test.csv", "15.3")

summonerId     4
puuid          4
matchId       20
matchData     20
dtype: int64


,summonerId,puuid,matchId,matchData
0,ykcp8Dwwqm33s1Q9_55qCmuJxx4Hi5cXR9yExXFPVQCYL1c,sdgTQOOEEBvxsKzEYUE4Vll5Pf8wC6So-27l9qDSrV5Gaf...,NA1_5226861599,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
1,ykcp8Dwwqm33s1Q9_55qCmuJxx4Hi5cXR9yExXFPVQCYL1c,sdgTQOOEEBvxsKzEYUE4Vll5Pf8wC6So-27l9qDSrV5Gaf...,NA1_5226826880,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
2,ykcp8Dwwqm33s1Q9_55qCmuJxx4Hi5cXR9yExXFPVQCYL1c,sdgTQOOEEBvxsKzEYUE4Vll5Pf8wC6So-27l9qDSrV5Gaf...,NA1_5226791216,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
3,ykcp8Dwwqm33s1Q9_55qCmuJxx4Hi5cXR9yExXFPVQCYL1c,sdgTQOOEEBvxsKzEYUE4Vll5Pf8wC6So-27l9qDSrV5Gaf...,NA1_5226766682,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
4,ykcp8Dwwqm33s1Q9_55qCmuJxx4Hi5cXR9yExXFPVQCYL1c,sdgTQOOEEBvxsKzEYUE4Vll5Pf8wC6So-27l9qDSrV5Gaf...,NA1_5226500664,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
5,JQjDqBq2YQsVkGNQ6--rmQrgQsTdCmqsqO5ooHLeEiqa-G...,jpqj_KsazPsSOhzGcCxZdF0rRuCWcVUFCqsB_A9p1FU5Wl...,NA1_5229648468,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
6,JQjDqBq2YQsVkGNQ6--rmQrgQsTdCmqsqO5ooHLeEiqa-G...,jpqj_KsazPsSOhzGcCxZdF0rRuCWcVUFCqsB_A9p1FU5Wl...,NA1_5224738854,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
7,JQjDqBq2YQsVkGNQ6--rmQrgQsTdCmqsqO5ooHLeEiqa-G...,jpqj_KsazPsSOhzGcCxZdF0rRuCWcVUFCqsB_A9p1FU5Wl...,NA1_5224663404,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
8,JQjDqBq2YQsVkGNQ6--rmQrgQsTdCmqsqO5ooHLeEiqa-G...,jpqj_KsazPsSOhzGcCxZdF0rRuCWcVUFCqsB_A9p1FU5Wl...,NA1_5224632058,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
9,JQjDqBq2YQsVkGNQ6--rmQrgQsTdCmqsqO5ooHLeEiqa-G...,jpqj_KsazPsSOhzGcCxZdF0rRuCWcVUFCqsB_A9p1FU5Wl...,NA1_5223808666,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."


Aggregate all data at the champion level:
Created 3 functions:
1. create_match_df(): 
    Takes a dataframe as input (for now it takes a csv file but can be easily changed to a df if needed). It then accesses the matchData column of that df and calls the create_df_participants() and create_df_teamdata() functions using the df it took from input (assigned to df_all_matches) as the argument for both functions. 
    - Next step would be to aggregate the data obtained from both helper functions at the champion level

2. create_df_participants():
    Takes the "matchData" column of df inputted to create_match_df() as input. It then extracts the value from the participant key (inside the info dictionary) and creates a new df where each participant in each game is a row and the columns are the data points from the matchdata API endpoint. This function flattens out the entire dictionary (so the keys from sub-dictionaries like "challenges" are columns). This also includes matchIds.

3. create_df_teamdata():  
    Takes the "matchData" column of df inputted to create_match_df() as input. It then extracts the value from the teams key (inside the info dictionary) and creates a new df where each team in each game is a row and the columns are the data points from the matchdata API endpoint (bans, objectives, etc.).
    IMPORTANT: The bans information is still nested in a list of dictionaries, where each dictionary is a ban.



In [ ]:
# Function to create a list with all match data
def create_match_df(dataframe_csv: str) -> list[dict]:
    # Convert csv to Dataframe (optional step)
    df_all_matches = pd.read_csv(dataframe_csv)
    # Parsing the JSON in each row so that matchData becomes a dictionary
    df_all_matches["matchData"] = df_all_matches["matchData"].apply(json.loads)
    # Call in function to create per participant dataframe
    df_participants = create_df_participants(df_all_matches)
    # Call in fucntion to create teamdata dataframe
    df_teams = create_df_teamdata(df_all_matches)
    # Placeholder return, to visualize better only return one variable at a time
    return df_participants, df_teams  
    
# Create a function to create a DataFrame from the participants column (i.e. a list of dictionaries)
def create_df_participants(df_all_matches: pd.DataFrame) -> pd.DataFrame:
    # Extract participants (assuming each match JSON has an "info" key with a "participants" list)
    df_all_matches["participants"] = df_all_matches["matchData"].apply(lambda m: m["info"]["participants"])
    # Explode the participants column so each participant becomes a row and remove index
    df_exploded = df_all_matches.explode("participants").reset_index(drop=True)
    # Flatten the participant dictionaries into a DataFrame using json_normalize
    df_participants = pd.json_normalize(df_exploded["participants"], sep = "_")
    # Add matchId from metadata (can use same logic to add other metadata columns)
    df_participants["matchId"] = df_exploded["matchData"].apply(lambda m: m["metadata"]["matchId"])
    return df_participants

# Create a function to create a DataFrame with data from each team (each row is a team), like bans and objectives
def create_df_teamdata(df_all_matches: pd.DataFrame) -> pd.DataFrame:
    # Extract the "teams" list (is a list of 2 dictionaries, one for each team)
    df_all_matches["teams"] = df_all_matches["matchData"].apply(lambda m: m["info"]["teams"])
    # Explode so each team (team 100 and team 200) becomes its own row and remove index
    df_exploded = df_all_matches.explode("teams").reset_index(drop=True)
    # Flatten the teams dictionaries (teamId, bans, objectives, etc.) into a DataFrame using json_normalize
    df_teams = pd.json_normalize(df_exploded["teams"], sep = "_")
    # Add matchId from metadata (can use same logic to add other metadata columns)
    df_teams["matchId"] = df_exploded["matchData"].apply(lambda m: m["metadata"]["matchId"])
    return df_teams

create_match_df("all_match_data_test.csv")

,bans,teamId,win,objectives_atakhan_first,objectives_atakhan_kills,objectives_baron_first,objectives_baron_kills,objectives_champion_first,objectives_champion_kills,objectives_dragon_first,objectives_dragon_kills,objectives_horde_first,objectives_horde_kills,objectives_inhibitor_first,objectives_inhibitor_kills,objectives_riftHerald_first,objectives_riftHerald_kills,objectives_tower_first,objectives_tower_kills,matchId
0,"[{'championId': 6, 'pickTurn': 1}, {'championI...",100,False,False,0,False,0,False,26,False,1,True,3,False,0,True,1,True,3,NA1_5226861599
1,"[{'championId': 893, 'pickTurn': 6}, {'champio...",200,True,True,1,True,1,True,42,True,4,False,3,True,1,False,0,False,8,NA1_5226861599
2,"[{'championId': 26, 'pickTurn': 1}, {'champion...",100,False,True,1,False,0,False,26,False,0,False,2,True,1,True,1,True,6,NA1_5226826880
3,"[{'championId': 131, 'pickTurn': 6}, {'champio...",200,True,False,0,True,1,True,23,True,3,True,4,False,1,False,0,False,9,NA1_5226826880
4,"[{'championId': 20, 'pickTurn': 1}, {'champion...",100,False,True,1,False,0,True,45,True,2,True,1,True,1,True,1,False,6,NA1_5226791216
5,"[{'championId': 29, 'pickTurn': 6}, {'champion...",200,True,False,0,True,2,False,32,False,3,False,5,False,2,False,0,True,10,NA1_5226791216
6,"[{'championId': 24, 'pickTurn': 1}, {'champion...",100,True,False,0,False,0,True,42,False,0,True,5,False,0,False,0,True,6,NA1_5226766682
7,"[{'championId': 517, 'pickTurn': 6}, {'champio...",200,False,False,0,False,0,False,17,True,2,False,1,False,0,False,0,False,1,NA1_5226766682
8,"[{'championId': 119, 'pickTurn': 1}, {'champio...",100,True,False,0,False,0,True,27,True,2,False,2,True,1,True,1,True,8,NA1_5226500664
9,"[{'championId': 157, 'pickTurn': 6}, {'champio...",200,False,False,0,False,0,False,16,False,1,True,4,False,0,False,0,False,1,NA1_5226500664
